## Read ADC information from the chip -- voltages
used to debug the chip measurements

In [1]:
import samna
import time

print(samna.__version__)
    
d= samna.device.get_unopened_devices()
dk = samna.device.open_device(d[0])
power_module = dk.get_power_module()
adc = power_module.get_digital_adc()

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x7)]) #digital current
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x5)]) #digital voltage
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x4)]) #digital v_shant
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x8)]) #digital raw power
time.sleep(2)
print(adc.read())

0.39.5.7+g0e6db422
[unifirm::adc::ina229::RegisterValue(address=7, value=1165216, timestamp=0)]
[unifirm::adc::ina229::RegisterValue(address=5, value=207840, timestamp=0)]
[unifirm::adc::ina229::RegisterValue(address=4, value=1117760, timestamp=0)]
[unifirm::adc::ina229::RegisterValue(address=8, value=56970, timestamp=0)]


## Basic power measurement -- Samna

In [2]:
import samna

# Open the device and connect to the power monitor.
board = samna.device.open_device("XyloAudio3TestBoard")
power_monitor = board.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = board.get_stop_watch()

model = board.get_model()
source = samna.graph.source_to(model.get_sink_node())
sink   = samna.graph.sink_from(model.get_source_node())

source.write([samna.xyloAudio3.event.WriteRegisterValue(address=0x168, data=0x0100_0000)])

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]
print(avgs)

[0.2684944046874686, 0.013561036032522603, 1.3764079539881398]


## Basic power measurement -- Rockpool

In [1]:
import warnings
warnings.filterwarnings("ignore")

import samna
print(f'Samna version {samna.__version__}')

import numpy as np
from rockpool.devices.xylo.syns65302 import config_from_specification as a3_config_from, mapper as a3_mapper, XyloSamna as a3_XyloSamna, XyloMonitor as a3_XyloMonitor
from rockpool.devices.xylo import find_xylo_hdks
xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()

xa3_node = ""
for version, xylo in zip(xylo_versions, xylo_hdks):
    if version == "syns65302":
        xa3_node = xylo

input_data = np.loadtxt('input_data.txt')
input_data = np.array(input_data, dtype=bool)

if xa3_node:
    a3_config = samna.xyloAudio3.configuration.XyloConfiguration()
    a3_config.operation_mode = samna.xyloAudio3.OperationMode.AcceleratedTime
    a3_config.input_source = samna.xyloAudio3.InputSource.SpikeEvents

    dt = 0.01
    Xmod = a3_XyloSamna(xa3_node, a3_config)    
    out, _, rec = Xmod.evolve(input_data, record = False, record_power = True)

    print(len(rec['io_power']))
    print(len(rec['analog_power']))
    print(len(rec['digital_power']))

    print(rec['io_power'])
    print(rec['analog_power'])
    print(rec['digital_power'])
    
    io_power = np.mean(rec['io_power'])
    analog = np.mean(rec['analog_power'])
    digital = np.mean(rec['digital_power'])


    print(f'XyloAudio 3\nAll IO:\t\t{io_power * 1000:.1f} mW\nAFE core:\t{analog * 1000:.1f} mW\nSNN core logic:\t{digital*1000:.1f} mW')


Samna version 0.39.5.7+g0e6db422
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a XyloAudio 3. Importing `rockpool.devices.xylo.syns65302`
1
0
0
0
[]
[]
[]
XyloAudio 3
All IO:		nan mW
AFE core:	nan mW
SNN core logic:	nan mW
